# Simulation study - Inference of tumour growth model in a virtual patient without treatment 

## Measure tumour volume in a virtual patient

In [1]:
import numpy as np

import erlotinib as erlo

# Define predictive model
path = erlo.ModelLibrary().tumour_growth_inhibition_model_koch_reparametrised()
mechanistic_model = erlo.PharmacodynamicModel(path)
error_models = [erlo.ConstantAndMultiplicativeGaussianErrorModel()]
predictive_model = erlo.PredictiveModel(mechanistic_model, error_models)

# Define a virtual patient
parameters = [
    0.22,  # Initial tumour volume
    3.76,  # Critical tumour volume
    0,     # Drug concentration (absence of treatment)
    0,     # Drug efficacy
    0.13,  # Tumour growth rate
    0.04,  # Base noise
    0.1]   # Relative noise

# Take virtual measurements
seed = 42
times = np.linspace(0, 30, num=10)
n_samples = 1
data = predictive_model.sample(parameters, times, n_samples, seed)

### Visualise virtual measurements

In [2]:
# Create scatter plot
fig = erlo.plots.PDTimeSeriesPlot()
fig.add_data(data, biomarker='myokit.tumour_volume', meas_key='Sample')
fig.set_axis_labels(xlabel=r'$\text{Time in day}$', ylabel=r'$\text{Tumour volume in cm}^3$')

# Show figure
fig.show()

**Figure 1:** Visualisation of the measured tumour growth in a virtual patient.

## Build prior predictive model

In [3]:
import pints

# Define prior distribution
log_prior_tumour_volume = pints.HalfCauchyLogPrior(location=1, scale=1)
log_prior_critical_volume = pints.HalfCauchyLogPrior(location=0, scale=1)
log_prior_drug_conc = pints.UniformLogPrior(0, 0.001)  # Fixed to 0 later
log_prior_kappa = pints.UniformLogPrior(0, 0.001)  # Fixed to 0 later
log_prior_lambda = pints.HalfCauchyLogPrior(location=0, scale=1)
log_prior_sigma_base = pints.HalfCauchyLogPrior(location=0, scale=1)
log_prior_sigma_rel = pints.HalfCauchyLogPrior(location=0, scale=0.1)
log_prior = pints.ComposedLogPrior(
    log_prior_tumour_volume, log_prior_critical_volume, log_prior_drug_conc, log_prior_kappa,
    log_prior_lambda, log_prior_sigma_base, log_prior_sigma_rel)

# Define prior predictive model and sample 1000 virtual patients
model = erlo.PriorPredictiveModel(predictive_model, log_prior)
n_samples = 1000
samples = model.sample(times, n_samples, seed)

# Visualise prior predictive model
fig = erlo.plots.PDPredictivePlot()
fig.add_prediction(data=samples, bulk_probs=[0.3, 0.6, 0.9])
fig.set_axis_labels(xlabel=r'$\text{Time in day}$', ylabel=r'$\text{Tumour volume in cm}^3$')
fig.show()

**Figure 2:** Prior predictive model of tumour volume measurement over time. The shaded areas estimate the 0.3, 0.6 and 0.9 bulk probability of the prior predictive distribution by sampling 1000 virtual "measurements" at each of the time points.

## Find MAP estimates for model parameters

In [4]:
# Set parameter names of mechanistic model (for convenience)
mechanistic_model.set_parameter_names(names={
        'myokit.tumour_volume': 'Tumour volume in cm^3',
        'myokit.critical_volume': 'Critical volume in cm^3',
        'myokit.drug_concentration': 'Drug concentration in mg/L',
        'myokit.kappa': 'Potency in L/mg/day',
        'myokit.lambda': 'Exponential growth rate in 1/day'})

# Create posterior
problem = erlo.ProblemModellingController(
    data, biom_keys=['Sample'])
problem.set_mechanistic_model(mechanistic_model)
problem.set_error_model(
    error_models=[pints.ConstantAndMultiplicativeGaussianLogLikelihood])
problem.fix_parameters(name_value_dict=dict({
    'Drug concentration in mg/L': 0,
    'Potency in L/mg/day': 0,
    'Noise param 2': 1}))
problem.set_log_prior(log_priors=[
    log_prior_tumour_volume, 
    log_prior_critical_volume,
    log_prior_lambda,
    log_prior_sigma_base,
    log_prior_sigma_rel])
log_posterior = problem.get_log_posteriors()

# Find maximum a posteriori probability estimates (MAP)
opt = erlo.OptimisationController(log_posterior)
opt.set_transform(transform=pints.LogTransformation(n_parameters=5))
map_estimates = opt.run(show_run_progress_bar=True)

8
['Tumour volume in cm^3', 'Critical volume in cm^3', 'Drug concentration in mg/L', 'Potency in L/mg/day', 'Exponential growth rate in 1/day', 'Noise param 1', 'Noise param 2', 'Noise param 3']


### Visualise optimisation results

In [5]:
fig = erlo.plots.ParameterEstimatePlot()
fig.add_data(map_estimates)

fig.show()

**Figure 3:** Maximum a posteriori (MAP) estimates of the model parameters. The y axis displays the estimated parameter value, and the x axis the corresponding individual.

## Find posterior probability distribution

In [8]:
# Set up sampling controller
sampler = erlo.SamplingController(log_posterior)
sampler.set_initial_parameters(data=map_estimates)
sampler.set_transform(transform=pints.LogTransformation(n_parameters=5))

# Run sampling
posterior_samples = sampler.run(n_iterations=4000, show_progress_bar=True)

### Visualise posterior samples

In [9]:
fig = erlo.plots.MarginalPosteriorPlot()
fig.add_data(data=posterior_samples, warm_up_iter=2000)

fig.show()

**Figure 4:** Marginal posterior distributions of model parameters. The y axis displays the sampled parameter value, and the x axis the binned number of samples for each individual.

## Posterior predictive checks

**WARNING:** This is currently not a posterior predictive check, but rather a check of the MAP estimates.

In [13]:
# Sample from predictive model
times = np.linspace(0, 30)
parameters = [0.24, 4.16, 0, 0, 0.12, 0.03, 0.06]
n_samples = 1000
samples = predictive_model.sample(parameters, times, n_samples)

# Visualise posterior predictive model
fig = erlo.plots.PDPredictivePlot()
fig.add_prediction(data=samples)
fig.add_data(data, biomarker='myokit.tumour_volume', meas_key='Sample')
fig.set_axis_labels(xlabel=r'$\text{Time in day}$', ylabel=r'$\text{Tumour volume in cm}^3$')
fig.show()

**Figure 5:** Predictive model constructed from the maximum a posteriori (MAP) estimates for individual 95. The shaded area illustrates the approximate 90% bulk probability of the predictive model constructed from 1000 samples for each of 50 equidistant time points, and the circles represent the measurements of individual 95.